In [4]:
from Horflip_equivariant_CNN_firstlayer import Conv2D_HF
from Horflip_equivariant_CNN_secondlayer import Conv2D_HF2

The following code snippet has been taken from https://www.tensorflow.org/tutorials/images/cnn and modified to be able to train a CNN model with some layers equivariant to horizontal flip in input space. The different strategies in train_model function force the network to be equivariant to horizontal flip for:

1) eq_first: the first CNN layer

2) eq_first_sec: first and second CNN layers

3) eq_first_sec_third: first, second and third CNN layers

4) standard: None of the layers

In [5]:
import numpy as np

import tensorflow as tf
from keras import backend as K

from tensorflow.keras import datasets, layers, models

from keras.utils.layer_utils import count_params

seed = 100

tf.random.set_seed(seed)
np.random.seed(seed)

(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

def train_model(strategy='standard'):
    """
    This function trains a model following one of these strategies:
    'standard': vanilla conv layers
    'keras_group_CNN': param groups=2, image and h-flipped version as input
    'hf_eq_layer': custom implementation of CNN layer equivariant to h-flip

    This function returns the acc in test set
    """
    
    tf.keras.backend.clear_session()

    if strategy == 'standard':
        conv_layer1 = layers.Conv2D
        conv_layer2 = layers.Conv2D
        conv_layer3 = layers.Conv2D      
        div1 = 1
        div2 = 1
        div3 = 1
    elif strategy == 'eq_first':
        conv_layer1 = Conv2D_HF
        conv_layer2 = layers.Conv2D
        conv_layer3 = layers.Conv2D
        div1 = 2
        div2 = 1
        div3 = 1
    elif strategy == 'eq_first_sec':
        conv_layer1 = Conv2D_HF
        conv_layer2 = Conv2D_HF2
        conv_layer3 = layers.Conv2D
        div1 = 2
        div2 = 2
        div3 = 1
    elif strategy == 'eq_first_sec_third':
        conv_layer1 = Conv2D_HF
        conv_layer2 = Conv2D_HF2
        conv_layer3 = Conv2D_HF2
        div1 = 2
        div2 = 2
        div3 = 2
    else:
        raise ValueError('Not implemented strategy')

    model = models.Sequential()
    model.add(conv_layer1(32//div1, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    model.add(layers.MaxPooling2D((2, 2)))    
    model.add(conv_layer2(64//div2, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))    
    model.add(conv_layer3(64//div3, (3, 3), activation='relu'))      

    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(10))

    model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
    
    _ = model.fit(train_images, train_labels, epochs=12, 
                        validation_data=(test_images, test_labels))    
    model.summary()

    _, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

    return test_acc

170500096/170498071 [==============================] - 2s 0us/step


In [6]:
def print_mean_std(strategy, samples):
  acc = []
  for i in range(samples):
    test_acc = train_model(strategy=strategy)
    acc.append(test_acc)
  print('the mean acc of method ' + strategy + ' is ' + str(np.array(acc).mean()))
  print('the std of acc of method ' + strategy + ' is ' + str(np.array(acc).std()))

In [7]:
print_mean_std('eq_first', 10)

Epoch 1/12
1563/1563 [==============================] - 13s 4ms/step - loss: 1.6765 - accuracy: 0.3826 - val_loss: 1.2016 - val_accuracy: 0.5720
Epoch 2/12
1563/1563 [==============================] - 5s 3ms/step - loss: 1.0966 - accuracy: 0.6119 - val_loss: 0.9824 - val_accuracy: 0.6542
Epoch 3/12
1563/1563 [==============================] - 5s 3ms/step - loss: 0.9206 - accuracy: 0.6746 - val_loss: 0.9521 - val_accuracy: 0.6714
Epoch 4/12
1563/1563 [==============================] - 5s 3ms/step - loss: 0.8330 - accuracy: 0.7081 - val_loss: 0.9080 - val_accuracy: 0.6872
Epoch 5/12
1563/1563 [==============================] - 5s 3ms/step - loss: 0.7600 - accuracy: 0.7333 - val_loss: 0.8559 - val_accuracy: 0.7069
Epoch 6/12
1563/1563 [==============================] - 6s 4ms/step - loss: 0.7003 - accuracy: 0.7525 - val_loss: 0.8426 - val_accuracy: 0.7077
Epoch 7/12
1563/1563 [==============================] - 5s 3ms/step - loss: 0.6570 - accuracy: 0.7721 - val_loss: 0.9007 - val_accuracy

In [8]:
print_mean_std('eq_first_sec', 10)

Epoch 1/12
1563/1563 [==============================] - 7s 4ms/step - loss: 1.6678 - accuracy: 0.3940 - val_loss: 1.1499 - val_accuracy: 0.5945
Epoch 2/12
1563/1563 [==============================] - 6s 4ms/step - loss: 1.0990 - accuracy: 0.6119 - val_loss: 1.0113 - val_accuracy: 0.6449
Epoch 3/12
1563/1563 [==============================] - 6s 4ms/step - loss: 0.9187 - accuracy: 0.6765 - val_loss: 0.9456 - val_accuracy: 0.6791
Epoch 4/12
1563/1563 [==============================] - 6s 4ms/step - loss: 0.8098 - accuracy: 0.7215 - val_loss: 0.9434 - val_accuracy: 0.6746
Epoch 5/12
1563/1563 [==============================] - 6s 4ms/step - loss: 0.7277 - accuracy: 0.7437 - val_loss: 0.9167 - val_accuracy: 0.6851
Epoch 6/12
1563/1563 [==============================] - 6s 4ms/step - loss: 0.6571 - accuracy: 0.7697 - val_loss: 0.9002 - val_accuracy: 0.6971
Epoch 7/12
1563/1563 [==============================] - 6s 4ms/step - loss: 0.6022 - accuracy: 0.7919 - val_loss: 0.9186 - val_accuracy:

In [9]:
print_mean_std('eq_first_sec_third', 10)

Epoch 1/12
1563/1563 [==============================] - 8s 5ms/step - loss: 1.6805 - accuracy: 0.3879 - val_loss: 1.1898 - val_accuracy: 0.5820
Epoch 2/12
1563/1563 [==============================] - 6s 4ms/step - loss: 1.1283 - accuracy: 0.6033 - val_loss: 1.0434 - val_accuracy: 0.6355
Epoch 3/12
1563/1563 [==============================] - 6s 4ms/step - loss: 0.9773 - accuracy: 0.6599 - val_loss: 1.0039 - val_accuracy: 0.6521
Epoch 4/12
1563/1563 [==============================] - 7s 4ms/step - loss: 0.9048 - accuracy: 0.6857 - val_loss: 0.9936 - val_accuracy: 0.6532
Epoch 5/12
1563/1563 [==============================] - 7s 4ms/step - loss: 0.8372 - accuracy: 0.7091 - val_loss: 0.9773 - val_accuracy: 0.6605
Epoch 6/12
1563/1563 [==============================] - 7s 4ms/step - loss: 0.7791 - accuracy: 0.7297 - val_loss: 0.9935 - val_accuracy: 0.6647
Epoch 7/12
1563/1563 [==============================] - 7s 4ms/step - loss: 0.7399 - accuracy: 0.7449 - val_loss: 0.9920 - val_accuracy:

In [10]:
print_mean_std('standard', 10)

Epoch 1/12
1563/1563 [==============================] - 6s 4ms/step - loss: 1.7775 - accuracy: 0.3425 - val_loss: 1.3290 - val_accuracy: 0.5173
Epoch 2/12
1563/1563 [==============================] - 5s 3ms/step - loss: 1.2190 - accuracy: 0.5681 - val_loss: 1.1043 - val_accuracy: 0.6078
Epoch 3/12
1563/1563 [==============================] - 5s 3ms/step - loss: 1.0343 - accuracy: 0.6327 - val_loss: 1.0289 - val_accuracy: 0.6397
Epoch 4/12
1563/1563 [==============================] - 5s 3ms/step - loss: 0.9287 - accuracy: 0.6740 - val_loss: 1.0706 - val_accuracy: 0.6280
Epoch 5/12
1563/1563 [==============================] - 5s 3ms/step - loss: 0.8489 - accuracy: 0.7004 - val_loss: 0.9437 - val_accuracy: 0.6694
Epoch 6/12
1563/1563 [==============================] - 6s 4ms/step - loss: 0.7811 - accuracy: 0.7258 - val_loss: 0.8956 - val_accuracy: 0.6841
Epoch 7/12
1563/1563 [==============================] - 6s 4ms/step - loss: 0.7412 - accuracy: 0.7395 - val_loss: 0.9180 - val_accuracy: